# Review plots and statistics for label refinement

In this notebook, we'll use tools in R to plot and compute summary statistics about our cell label refinements.

## Install packages

This package includes helper functions for plot colorsets

In [1]:
devtools::install_github("hypercompetent/colorway", upgrade = "never", quiet = TRUE)

This package includes helpers for plot types, including river/sankey plots

In [4]:
devtools::install_github("alleninstitute/scrattch.vis", upgrade = "always", quiet = TRUE)

## Load libraries

`colorway`: ggplot colorsets  
`cowplot`: Plot grid arrangement  
`hise`: The R SDK for HISE  
`data.table`: Efficient implementation of data frames  
`dplyr`: Manipulation of data frames  
`ggplot2`: Plotting using the grammar of graphics  
`ggrastr`: Rasterized plotting to help with large UMAPs  
`purrr`: functional programming tools for R  
`furrr`: paralellization of `purrr` using `futures`  
`scrattch.vis`: Plotting helper functions

In [5]:
quiet_library <- function(...) { suppressPackageStartupMessages(library(...)) }

quiet_library(colorway)
quiet_library(cowplot)
quiet_library(hise)
quiet_library(data.table)
quiet_library(dplyr)
quiet_library(ggplot2)
quiet_library(ggrastr)
quiet_library(purrr)
quiet_library(furrr)
quiet_library(scrattch.vis)

In [6]:
plan(multisession, workers = 12)

In [7]:
if(!dir.exists("output")) {
    dir.create("output")
}

## Read cell type hierarchy from HISE

Needed for cases where we forgot to back-propagate L2 and L1 types based on final AIFI_L3 designations

In [8]:
hierarchy_uuid <- "1a44252c-8cab-4c8f-92c9-d8f3af633790"
hise_res <- cacheFiles(list(hierarchy_uuid))
hierarchy_file <- list.files(paste0("cache/", hierarchy_uuid), full.names = TRUE)
hierarchy_df <- read.csv(hierarchy_file, row.names = 1)

## Helper functions

These functions pull our previously generated results and provide paths to the files after retrieving them.

In [9]:
cache_path_uuid <- function(uuid) {
    cache_dir = paste0('cache/', uuid)
    if (!dir.exists(cache_dir)) {
        cacheFiles(list(uuid))
    }
    list.files(cache_dir, full.names = TRUE)
}

In [10]:
cache_tar_path_uuid <- function(uuid) {
    cache_dir = paste0('cache/', uuid)
    if (!dir.exists(cache_dir)) {
        cacheFiles(list(uuid))
    }
    if (!dir.exists('output/review')) {
        dir.create('output/review')
    }
    tarfile = list.files(cache_dir, full.names = TRUE)
    old_rev_files = list.files('output/review', full.names = TRUE)
    system(paste("tar -xf", tarfile))
    new_rev_files = list.files('output/review', full.names = TRUE)
    setdiff(new_rev_files, old_rev_files)
}

## Plotting functions

In [11]:
format_perc <- function(x, dec = 2) {
    format(round(x * 100, 2), nsmall = dec)
}

In [12]:
plot_umap <- function(meta_df, color_by = "AIFI_L2", color_name = NULL) {
    if(is.null(color_name)) {
        color_name <- color_by
    }
    color_by = rlang::parse_expr(color_by)

    meta_df <- meta_df[sample(1:nrow(meta_df), nrow(meta_df)),]
    
    p <- ggplot() +
      geom_point_rast(
          data = meta_df,
          aes(x = umap_1,
              y = umap_2,
              color = !!color_by),
          size = 0.02
      ) +
      scale_color_varibow() +
      large_guides() +
      theme_bw() +
      theme(axis.ticks = element_blank(),
            legend.position = "bottom")
    p
}

In [13]:
plot_markers <- function(marker_df, level = "AIFI_L2") {
    level = rlang::parse_expr(level)
    
    ggplot() +
      geom_point(
          data = marker_df,
          aes(x = gene,
              y = !!level,
              size = gene_frac,
              fill = log(gene_mean + 1)),
          pch = 21
      ) +
      scale_size_area() +
      scale_fill_gradientn(
          colors = c("black", "darkred", "red", "orangered", "orange")
      ) +
      theme_bw() +
      theme(axis.ticks = element_blank(),
            axis.text.x = element_text(angle = 90, hjust = 1, vjust = 0.3))
}

In [14]:
get_low_counts <- function(meta, column, collapse_n) {
    column <- rlang::parse_expr(column)
    meta %>%
      group_by(!!column) %>%
      summarise(n_cells = n(), .groups = "keep") %>%
      filter(n_cells <= collapse_n) %>%
      select(!!column) %>%
      unlist()
}

In [15]:
plot_l2_river <- function(meta, l2_colors, collapse_n = 10) {
    plot_meta <- meta %>%
      select(original_L2, predicted_AIFI_L2, AIFI_L2)
    
    if(!is.null(collapse_n)) {
        original_drop <- get_low_counts(plot_meta, "original_L2", collapse_n)
        predicted_drop <- get_low_counts(plot_meta, "predicted_AIFI_L2", collapse_n)
        plot_meta <- plot_meta %>%
          mutate(original_L2 = ifelse(original_L2 %in% original_drop, "other", original_L2),
                 predicted_AIFI_L2 = ifelse(predicted_AIFI_L2 %in% predicted_drop, "other", predicted_AIFI_L2))
    }
    
    l2_plot_data <- plot_meta %>%
      left_join(rename(l2_colors, original_L2 = L2, original_L2_id = id, original_L2_color = color),
                by = "original_L2") %>%
      left_join(rename(l2_colors, predicted_AIFI_L2 = L2, predicted_AIFI_L2_id = id, predicted_AIFI_L2_color = color),
                by = "predicted_AIFI_L2") %>%
      left_join(rename(l2_colors, AIFI_L2 = L2, AIFI_L2_id = id, AIFI_L2_color = color),
                by = "AIFI_L2") %>%
      rename(original_L2_label = original_L2,
             predicted_AIFI_L2_label = predicted_AIFI_L2,
             AIFI_L2_label = AIFI_L2)

    build_river_plot(
        anno = l2_plot_data,
        grouping = c("original_L2", "predicted_AIFI_L2", "AIFI_L2"),
        label_pos = c("left", "center", "right"),
        fill_group = "predicted_AIFI_L2"
    ) + 
        scale_x_continuous(expand = c(0.3, 0.3)) +
        ggtitle("L2 Original -> Predicted -> Final") +
        theme(plot.background = element_rect(fill = "white"))
}

In [16]:
plot_l3_river <- function(meta, l3_colors, collapse_n = 10) {
    plot_meta <- meta %>%
        select(original_L3, predicted_AIFI_L3, AIFI_L3)
    
    if(!is.null(collapse_n)) {
        original_drop <- get_low_counts(plot_meta, "original_L3", collapse_n)
        predicted_drop <- get_low_counts(plot_meta, "predicted_AIFI_L3", collapse_n)
        plot_meta <- plot_meta %>%
          mutate(original_L3 = ifelse(original_L3 %in% original_drop, "other", original_L3),
                 predicted_AIFI_L3 = ifelse(predicted_AIFI_L3 %in% predicted_drop, "other", predicted_AIFI_L3))
    }
    
    l3_plot_data <- plot_meta %>%
      left_join(rename(l3_colors, original_L3 = L3, original_L3_id = id, original_L3_color = color),
                by = "original_L3") %>%
      left_join(rename(l3_colors, predicted_AIFI_L3 = L3, predicted_AIFI_L3_id = id, predicted_AIFI_L3_color = color),
                by = "predicted_AIFI_L3") %>%
      left_join(rename(l3_colors, AIFI_L3 = L3, AIFI_L3_id = id, AIFI_L3_color = color),
                by = "AIFI_L3") %>%
      rename(original_L3_label = original_L3,
             predicted_AIFI_L3_label = predicted_AIFI_L3,
             AIFI_L3_label = AIFI_L3)

    build_river_plot(
        anno = l3_plot_data,
        grouping = c("original_L3", "predicted_AIFI_L3", "AIFI_L3"),
        label_pos = c("left", "center", "right"),
        fill_group = "predicted_AIFI_L3"
    ) + 
        scale_x_continuous(expand = c(0.3, 0.3)) +
        ggtitle("L3 Original -> Predicted -> Final") +
        theme(plot.background = element_rect(fill = "white"))
}

## Previous, non-tracked version of cell labels

We previously performed cell labeling in a way that wasn't tracked in our CertPro system. We'll compare our new results to these original results and make sure they're similar.

In [17]:
label_uuid <- "3868592c-0087-4ed8-98b2-4bf1b8676111"
res <- cacheFiles(list(label_uuid))
label_parquet <- list.files(paste0("cache/",label_uuid), full.names = TRUE)
label_csv <- sub("parquet","csv",label_parquet)
label_csv <- basename(label_csv)

Convert from parquet to csv for easy reading in R because our IDEs are having trouble with the `arrow` package:

In [18]:
system_call <- paste0(
    "python -c \'",
    "import pandas; ",
    "df = pandas.read_parquet(\"",label_parquet,"\");",
    "df = df[[\"barcodes\",\"AIFI_L1\",\"AIFI_L2\",\"AIFI_L3\"]];",
    "df.to_csv(\"",label_csv,"\")",
    "\'"
)

In [19]:
system_call

[1] "python -c 'import pandas; df = pandas.read_parquet(\"cache/3868592c-0087-4ed8-98b2-4bf1b8676111/all_imm_health_obs.parquet\");df = df[[\"barcodes\",\"AIFI_L1\",\"AIFI_L2\",\"AIFI_L3\"]];df.to_csv(\"all_imm_health_obs.csv\")'"

In [20]:
system(system_call)

Python version information in case we need it for reproducibility:

In [21]:
system("python --version", intern = TRUE)

[1] "Python 3.10.13"

In [22]:
system("python -c 'import pandas; print(pandas.__version__)'", intern = TRUE)

[1] "2.1.4"

In [23]:
og_labels <- fread(label_csv, header = TRUE)
og_labels <- og_labels[,V1:=NULL]

In [24]:
og_labels <- as.data.frame(og_labels)

In [25]:
names(og_labels) <- sub("AIFI", "original", names(og_labels))

In [26]:
nrow(og_labels)

[1] 13846464

In [27]:
head(og_labels)

,barcodes,original_L1,original_L2,original_L3
,<chr>,<chr>,<chr>,<chr>
1,05ea9806794211eb93b836d1cb6129eb,DC,cDC1,cDC1
2,e225c914794011eb9282e2ceeb91ba52,DC,cDC1,cDC1
3,b1379eae795411eb958b0245821e6993,DC,cDC1,cDC1
4,b13d3a8a795411eb958b0245821e6993,DC,cDC1,cDC1
5,b1430d16795411eb958b0245821e6993,DC,cDC1,cDC1
6,7f926876794e11eb80b0ca7613f64db1,DC,cDC1,cDC1


In [28]:
og_split <- split(og_labels, "original_L2")

## Generate consistent colors for plotting

In [29]:
l2_colors <- og_labels %>%
  select(original_L2) %>%
  unique() %>%
  arrange(original_L2) %>%
  mutate(id = 1:n(), 
         color = varibow(n())) %>%
  rename(L2 = original_L2)
l2_colors <- rbind(l2_colors, data.frame(L2 = "other", id = max(l2_colors$id) + 1, color = "#000000") )

In [30]:
l3_colors <- og_labels %>%
  select(original_L3) %>%
  unique() %>%
  arrange(original_L3) %>%
  mutate(id = 1:n(),
         color = varibow(n())) %>%
  rename(L3 = original_L3)
l3_colors <- rbind(l3_colors, data.frame(L3 = "other", id = max(l3_colors$id) + 1, color = "#000000") )

## Large cell sets: decompress results

For large cell sets, review data were stored in .tar.gz files. We'll retrieve these from HISE and extract data for plot generation.

In [31]:
large_tar_uuids <- list(
    monocyte_markers = '4d2707b8-634a-4057-8a99-a810966457ec',
    monoctye_labels = 'a205f299-1e28-4cd6-a19b-40e6433fa21b',
    nk_cells = '73fd91b2-0ef5-429e-877e-b7e9f8748d34',
    t_cd4_naive = '3a562e35-c698-4d11-a1b1-c609ce94f65d',
    t_cd4_memory = '016663fa-ea76-4cbc-b7f8-81ac7957dc24',
    t_cd8_memory = 'eb1c31d3-5288-4092-84d0-a719b9234839'
)

In [32]:
large_paths <- map2(
    large_tar_uuids, names(large_tar_uuids),
    function(uuid, group_name) {
        res <- cacheFiles(list(uuid))
        cache_dir <- paste0("cache/", uuid)
        tar_file <- list.files(cache_dir, pattern = ".tar.gz", full.names = TRUE)
        system_call <- paste('tar -xzf', tar_file)
        system(system_call)

        group_files <- list.files("output/review", full.names = TRUE)
        group_path <- paste0("data/",group_name)

        if(!dir.exists(group_path)) {
            dir.create(group_path, recursive = TRUE)
        }
        move_files <- sub("output/review", group_path, group_files)
        file.rename(group_files, move_files)

        list(
            meta = list.files(group_path, pattern = "review_meta", full.names = TRUE),
            l2_markers = list.files(group_path, pattern = "L2_review_markers", full.names = TRUE),
            l3_markers = list.files(group_path, pattern = "L3_review_markers", full.names = TRUE)
        )
    }
)

## Small cell sets

For smaller cell subsets, data were stored as .csv files for plot generation

In [33]:
small_uuids <- list(
    asdc = list(
        meta = 'cce70b23-5728-4588-963e-8f0afafa3943',
        l2_markers = 'fa63cbdd-2652-4c4e-957b-9570fc9c07d4',
        l3_markers = 'dbc20c14-5167-4e3f-a3bf-d9220dcfbe69'),
    b_memory_cells = list(
        meta = 'a712dc74-43b3-4dff-927e-d018dd369f9b',
        l2_markers = '9f584726-b096-44fc-91fb-8a73990fad9f',
        l3_markers = '6f79ee5a-7139-45d7-937d-42560209f3be'),
    b_naive_cells = list(
        meta = '3a1b1129-65ca-4f17-bc0f-2a53ac2ae963',
        l2_markers = 'd2ed48a3-55b4-4d96-96ec-c18724be1dd0',
        l3_markers = '785537bb-e651-454d-b332-4a971dc7cc87'),
    dc = list(
        meta = '63287a4b-adf0-42fc-8a75-37223d2f97de',
        l2_markers = '7b88180c-c403-4b67-a3a7-8d7577306516',
        l3_markers = '0474e3de-fc64-4b76-af50-f8da731a372d'),
    gdt_cd8aa = list(
        meta = 'cd513dd2-a72f-48e6-85b7-bad4676cb13f',
        l2_markers = '460daa3f-18be-4db3-a758-14d5e7401067',
        l3_markers = '9c513b26-4852-4f13-8cf9-d6f15b05e76a'),
    treg = list(
        meta = 'aab9cd47-29cd-4be4-bb86-2ed511b3d4e2',
        l2_markers = '6471c399-6248-4332-a96f-bb6ffb49cc82',
        l3_markers = 'a5c244f4-108f-4495-82de-8e774b73de60'),
    progen_prolif = list(
        meta = '8226e775-8f4d-452e-b81e-3f3fb4a70d1e',
        l2_markers = 'a6296c2b-ccf5-40b5-b3c9-2a0820532cc6',
        l3_markers = '7fbbd0f0-2162-40b8-8511-7380f8657280'),
    eryth_platelet = list(
        meta = 'e5c881d6-0576-4067-b624-4e984fe97900',
        l2_markers = '754d78fe-5ca5-49c4-aa26-33e06ab76fd1',
        l3_markers = '1f0160f4-8e32-4f98-8960-b7086fcb0cee'),
    plasma_ilc = list(
        meta = 'b3636d73-58c9-4972-8fb9-1cb51439c377',
        l2_markers = 'b5034de4-5650-4c8f-ae25-c8cb0449463d',
        l3_markers = '68d14aa0-a632-4f53-8c8a-5c90441b20fe'),
    dnt_mait = list(
        meta = '4ce56aca-03a9-46b2-9a32-924a35602fce',
        l2_markers = 'bcaf5cc3-9255-4223-bd84-95795853dd2b',
        l3_markers = 'f1dd5a2b-38e2-4787-a2d3-d2a49cbf3353'),
    cd8_naive = list(
        meta = '45668c5c-1a48-4fae-9d59-fa566063263e',
        l2_markers = 'ed30d2e7-5c8e-4477-bd9b-c378d262a531',
        l3_markers = '83046eb4-730a-4e4d-af50-3fc7d03f5942')
)

In [34]:
small_paths <- future_map(
    small_uuids,
    function(uuid_set) {
        map(uuid_set, cache_path_uuid)
    }
)

In [35]:
all_paths <- c(small_paths, large_paths)

In [36]:
all_dfs <- future_map(
    all_paths,
    function(path_set) {
        dts <- map(path_set, fread)
        map(dts, as.data.frame)
    }
)

In [37]:
all_dfs <- map(
    all_dfs,
    function(df_set) {
        df_set$meta <- df_set$meta %>%
          left_join(og_labels, by = "barcodes")
        df_set
    }
)

Fix categorical label names

In [38]:
all_dfs <- map(
    all_dfs,
    function(df_set) {
        map(df_set,
            function(df) {
                if("AIFI_L3" %in% names(df)) {
                    df$AIFI_L3 <- gsub("', .+","",df$AIFI_L3)
                    df$AIFI_L3 <- sub("\\['","",df$AIFI_L3)
                }
                df
            })
    }
)

Fix missing hierarchical assignment

In [39]:
all_dfs <- map(
    all_dfs,
    function(df_set) {
        meta <- df_set$meta
        if(!"predicted_AIFI_L2" %in% names(meta)) {
            meta <- meta %>%
              rename(predicted_AIFI_L1 = AIFI_L1,
                     predicted_AIFI_L2 = AIFI_L2) %>%
              left_join(hierarchy_df, by = "AIFI_L3")
            df_set$meta <- meta
        }
        df_set
    }
)

In [40]:
walk2(
    all_dfs, names(all_dfs),
    function(df_set, set_name) {

        orig_l2_umap <- plot_umap(df_set$meta, "original_L2")
        pred_l2_umap <- plot_umap(df_set$meta, "predicted_AIFI_L2")
        l2_umap <- plot_umap(df_set$meta, "AIFI_L2")

        orig_l3_umap <- plot_umap(df_set$meta, "original_L3")
        pred_l3_umap <- plot_umap(df_set$meta, "predicted_AIFI_L3")
        l3_umap <- plot_umap(df_set$meta, "AIFI_L3")

        l2_river <- plot_l2_river(df_set$meta, l2_colors)
        l3_river <- plot_l3_river(df_set$meta, l3_colors)

        l2_marker <- plot_markers(df_set$l2_markers, "AIFI_L2")
        l3_marker <- plot_markers(df_set$l3_markers, "AIFI_L3")

        umap_grid <- plot_grid(orig_l2_umap, pred_l2_umap, l2_umap,
                               orig_l3_umap, pred_l3_umap, l3_umap,
                               ncol = 3, nrow = 2,
                               align = "hv")
        umap_file <- paste0("output/diha_", set_name, "_review_umaps_",Sys.Date(),".png")
        ggsave(
            umap_file,
            umap_grid,
            width = 20, height = 18
        )
        
        river_grid <- plot_grid(l2_river, l3_river,
                                ncol = 2, nrow = 1)
        marker_grid <- plot_grid(l2_marker, l3_marker,
                                 ncol = 1, nrow = 2,
                                 rel_heights = c(0.5, 1))
        other_grid <- plot_grid(marker_grid, river_grid,
                                ncol = 2, nrow = 1,
                                rel_widths = c(0.7,1))
        
        other_file <- paste0("output/diha_", set_name, "_review_plots_",Sys.Date(),".png")
        ggsave(
            other_file,
            other_grid,
            width = 24, height = 8
        )
    }
)

Warning message:
“Removed 625 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 625 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_text()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_text()`).”
Warning message:
“Removed 3683 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 3683 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_text()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_text()`).”
Warning message:
“Removed 13144 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 13144 rows containing missing values (`geom_point()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_text()`).”
Warning message:
“Removed 1 rows containing missing values (`geom_text()`).”
Warning message:
“Removed 981 rows containing missin

In [41]:
sessionInfo()

R version 4.3.2 (2023-10-31)
Platform: x86_64-conda-linux-gnu (64-bit)
Running under: Ubuntu 20.04.6 LTS

Matrix products: default
BLAS/LAPACK: /opt/conda/lib/libopenblasp-r0.3.25.so;  LAPACK version 3.11.0

locale:
 [1] LC_CTYPE=C.UTF-8       LC_NUMERIC=C           LC_TIME=C.UTF-8       
 [4] LC_COLLATE=C.UTF-8     LC_MONETARY=C.UTF-8    LC_MESSAGES=C.UTF-8   
 [7] LC_PAPER=C.UTF-8       LC_NAME=C              LC_ADDRESS=C          
[10] LC_TELEPHONE=C         LC_MEASUREMENT=C.UTF-8 LC_IDENTIFICATION=C   

time zone: Etc/UTC
tzcode source: system (glibc)

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] scrattch.vis_0.1.000 ggdendro_0.2.0       ggbeeswarm_0.7.2    
 [4] furrr_0.3.1          future_1.33.1        purrr_1.0.2         
 [7] ggrastr_1.0.2        ggplot2_3.5.0        dplyr_1.1.4         
[10] data.table_1.15.0    hise_2.16.0          cowplot_1.1.2       
[13] colorway_0.2.0      

loaded via a n